In [1]:
import pandas as pd
import numpy as np

# Load data
csv_file_path = '/Users/balmeru/Downloads/1985.csv'
df = pd.read_csv(csv_file_path)

# Ensure required columns are not null
columns_to_check = ['ajexdi', 'prccd', 'trfd']
df.dropna(subset=columns_to_check, how='any', inplace=True)

# Convert datadate to datetime and sort
df['datadate'] = pd.to_datetime(df['datadate'], errors='coerce')
df.sort_values(by=['tic', 'datadate'], inplace=True)

# Calculate adjusted close prices
df['adjusted_close'] = df['prccd'] / df['ajexdi']
df['adjusted_close_prior'] = df.groupby('tic')['adjusted_close'].shift(1)
df['trfd_prior'] = df.groupby('tic')['trfd'].shift(1)

# Initialize daily_return with np.nan
df['daily_return'] = np.nan

# Calculate daily returns
df['daily_return'] = (
    ((df['adjusted_close'] * df['trfd']) /
     (df['adjusted_close_prior'] * df['trfd_prior'])) 
) 

# Set daily_return to NaN if the gap between days is too large
df['day_diff'] = df.groupby('tic')['datadate'].diff().dt.days
df.loc[df['day_diff'] > 5, 'daily_return'] = np.nan

# Create a pivot table of daily returns
adjusted_daily_returns = df.pivot_table(index='datadate', columns='tic', values='daily_return')


# Calculate weekly returns
all_weekdays = pd.date_range(start=adjusted_daily_returns.index.min(), end=adjusted_daily_returns.index.max(), freq='B')
df_reindexed = adjusted_daily_returns.reindex(all_weekdays)

all_mondays = pd.date_range(start=df_reindexed.index.min(), end=df_reindexed.index.max(), freq='W-MON')
weekly_pivot_df = pd.DataFrame(data=np.nan, index=all_mondays, columns=df_reindexed.columns)
ticker_start_dates = df_reindexed.apply(lambda x: x.first_valid_index(), axis=0)

for i, monday in enumerate(all_mondays):
    start_date = monday
    if i < len(all_mondays) - 1:
        end_date = all_mondays[i + 1] - pd.DateOffset(days=1)
    else:
        end_date = df_reindexed.index.max()
    
    weekly_data = df_reindexed.loc[start_date:end_date]
    
    for ticker in df_reindexed.columns:
        if monday < ticker_start_dates[ticker]:
            weekly_pivot_df.loc[monday, ticker] = np.nan
        else:
            if weekly_data[ticker].isnull().all():
                weekly_pivot_df.loc[monday, ticker] = np.nan
            else:
                # Calculate weekly return if there are valid daily returns in the week
                valid_weekly_data = weekly_data[ticker].dropna()
                if not valid_weekly_data.empty:
                    weekly_return = valid_weekly_data.prod() - 1
                    weekly_pivot_df.loc[monday, ticker] = weekly_return * 100
                else:
                    weekly_pivot_df.loc[monday, ticker] = np.nan

weekly_pivot_df.dropna(axis=1, how='all', inplace=True)
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

# Save the weekly pivot table to CSV
weekly_pivot_path = "/Users/balmeru/Downloads/weekly_pivot_1985.csv"
weekly_pivot_df.to_csv(weekly_pivot_path)
print(f"Weekly Pivot Table saved to: {weekly_pivot_path}")
print("Weekly Returns Pivot Table (first 10 rows):")
print(weekly_pivot_df.head(10))


Weekly Pivot Table saved to: /Users/balmeru/Downloads/weekly_pivot_1985.csv
Weekly Returns Pivot Table (first 10 rows):
tic         0223B  0485B  0491B  3234B  3614B  3ACDGE  3ACKH  3ACTRQ  3ADLN  \
1984-12-10   0.93  -4.35 -19.13   0.89   5.90    9.83   1.96   -2.78  -2.44   
1984-12-17   2.78   9.09   2.15  -0.00   2.99   20.89   1.54    2.86  -0.00   
1984-12-24   0.00   0.00  10.53  -1.32  -0.00  -17.28   0.76    2.78   0.00   
1984-12-31  -1.80   0.00  -1.90   1.33  -3.55   -4.49  -0.00    0.00  -0.00   
1985-01-07   0.92   0.00  -2.91   0.00   4.01   12.50   2.26    6.76  -2.50   
1985-01-14   1.82  18.75  25.00   7.89   4.18   13.89  -0.37   -1.27  -0.00   
1985-01-21   0.00  -3.51  -8.00   8.54   8.02   -1.21   7.01   15.38   5.13   
1985-01-28   7.14   9.09 -11.30   5.62  -4.57   -7.41   0.69   10.00  -2.44   
1985-02-04   3.33   0.00   0.98   2.13  10.18    9.32   3.92    6.06  -0.00   
1985-02-11  -3.23  10.00  -6.80  -2.08  -2.72    6.11  -1.33   -0.95   5.00   

tic       